## Benchmark Lorenz 63 linear and nonlinear filters

In this notebook, we are interested in the sequential inference 



References: 


[1] Evensen, G., 1994. Sequential data assimilation with a nonlinear quasi‐geostrophic model using Monte Carlo methods to forecast error statistics. Journal of Geophysical Research: Oceans, 99(C5), pp.10143-10162.

[2] Asch, M., Bocquet, M. and Nodet, M., 2016. Data assimilation: methods, algorithms, and applications. Society for Industrial and Applied Mathematics.

[3] Bishop, C.H., Etherton, B.J. and Majumdar, S.J., 2001. Adaptive sampling with the ensemble transform Kalman filter. Part I: Theoretical aspects. Monthly weather review, 129(3), pp.420-436. 

[4] Lorenz, E.N., 1963. Deterministic nonperiodic flow. Journal of atmospheric sciences, 20(2), pp.130-141.

[5] Spantini, A., Baptista, R. and Marzouk, Y., 2019. Coupling techniques for nonlinear ensemble filtering. arXiv preprint arXiv:1907.00389.

### The basic steps
To carry out sequential inference in `AdaptiveTransportMap`, we need to carry out a few basic steps:
* **Specify the problem**: Define the state-space model: initial condition, dynamical and observation models (including process and observation noise)
* **Specify the inflation parameters**: Determine the levels of covariance inflation to properly balance the dynamical system and the observations from the truth system
* **Specify the filter**: Choose the ensemble filter to assimilate the observations in the state estimate
* **Perform the sequential inference**: Perform the sequential inference

We will go through all of these here.

In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using OrdinaryDiffEq
using JLD

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been

In [2]:
using DelimitedFiles

Load some packages to make nice figures

In [3]:
using Plots
default(tickfont = font("CMU Serif", 9), 
        titlefont = font("CMU Serif", 14), 
        guidefont = font("CMU Serif", 12),
        legendfont = font("CMU Serif", 10),
        grid = false)
# Plots.font("sans-serif")
# clibrary(:colorbrewer)
# gr()
pyplot()

using LaTeXStrings
# PyPlot.rc("text", usetex = "true")
# rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
# rcParams["text.usetex"] = true;
PyPlot.rc("font", family = "CMU Serif")
PyPlot.matplotlib[:rc]("mathtext",fontset="cm")        #computer modern font 
PyPlot.matplotlib[:rc]("font",family="serif",size=12)

using ColorSchemes

The Lorenz-63  model is a three dimensional system that models the atmospheric convection [4]. This system is a classical benchmark problem in data assimilation. The state $\boldsymbol{x} = (x_1, x_2, x_3)$ is governed by the following set of ordinary differential equations:

\begin{equation}
\begin{aligned}
&\frac{\mathrm{d} x_1}{\mathrm{d} t}=\sigma(x_2-x_1)\\
&\frac{\mathrm{d} x_2}{\mathrm{d} t}=x_1(\rho-x_2)-x_2\\
&\frac{\mathrm{d} x_3}{\mathrm{d} t}=x_1 x_2-\beta x_3,
\end{aligned}
\end{equation}

where $\sigma = 10, \beta = 8/3, \rho = 28$. For these values, the system is chaotic and behaves like a strange attractor. We integrate this system of ODEs with time step $\Delta t_{dyn} = 0.05$. The state is fully observed $h(t,\boldsymbol{x}) = \boldsymbol{x}$ with $\Delta t_{obs}=0.1$. The initial distribution $\pi_{\mathsf{X}_0}$ is the standard Gaussian. The process noise is Gaussian with zero mean and covariance $10^{-4}\boldsymbol{I}_3$. The measurement noise has a Gaussian distribution with zero mean and covariance $\theta^2\boldsymbol{I}_3$ where $\theta^2 = 4.0$.

In [6]:
path = "/media/mat/HDD/AdaptiveTransportMap/notebooks/lorenz63/data/"

β_array = collect(0.95:0.01:1.05)
Ne_array = [10, 20, 40, 60, 100, 200, 400, 600]

model, data = setup_lorenz63(path, Ne_array);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Ne 10 RMSE: 0.5928069316894501


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Ne 20 RMSE: 0.5444225927797981


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


...

Ne 200 RMSE: 0.48105668064964185


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


Ne 400 RMSE: 0.48282592384478173


Progress: 100%|█████████████████████████████████████████| Time: 0:00:20


Ne 600 RMSE: 0.4846248170532807


In [7]:
# save(path*"lorenz63_data.jld", "data", data)

In [8]:
data = load(path*"lorenz63_data.jld", "data")

SyntheticData([0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0  …  599.1, 599.2, 599.3000000000001, 599.4, 599.5, 599.6, 599.7, 599.8000000000001, 599.9, 600.0], [-0.22950363471447657, 0.9216676448369137, -0.2969466755253305], [0.5493777232168378 1.9325334655339608 … 5.846018274793296 3.9203793494802213; 1.3470852152857304 4.233634108048278 … 2.4991778332625634 3.3036939755885326; -0.20782547004567642 0.12823736565833915 … 28.37631893621292 22.834861253310663], [1.884515687964999 0.15701426740781987 … 4.525541006447913 3.4631303339151946; -0.05946240332624342 2.8307715835113862 … 4.382540618269552 3.697992563381411; -1.8223964161767956 -0.25316028804554913 … 29.750657308982625 26.336548121393665])

In [9]:
Nx = 3
Ny = 3
Δtdyn = 0.05
Δtobs = 0.1

σx = 1e-6
σy = 2.0

ϵx = AdditiveInflation(Nx, zeros(Nx), σx)
ϵy = AdditiveInflation(Ny, zeros(Ny), σy)
tspinup = 200.0
Tspinup = 2000
tmetric = 400.0
Tmetric = 4000
t0 = 0.0
tf = 600.0
Tf = 6000

Tburn = 2000
Tstep = 4000
Tspinup = 2000

f = lorenz63!
h(x, t) = x
# Create a local version of the observation operator
h(x, t, idx) = x[idx]
F = StateSpace(lorenz63!, h)

model = Model(Nx, Ny, Δtdyn, Δtobs, ϵx, ϵy, MvNormal(zeros(Nx), Matrix(1.0*I, Nx, Nx)), Tburn, Tstep, Tspinup, F);

Benchmark for the Lorenz-63 problem with the sequential stochastic EnkF

In [10]:
metric_enkf = benchmark_lorenz63(model, data, path, Ne_array, β_array);

(Ne, β) = (10, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


  0.482059 seconds (3.65 M allocations: 331.523 MiB, 8.14% gc time)
Ne = 10
Ne 10& β 0.95 RMSE: 0.6663970586950805
(Ne, β) = (10, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


  0.434502 seconds (3.43 M allocations: 320.851 MiB, 8.87% gc time)
Ne = 10
Ne 10& β 0.96 RMSE: 4.2005659601223435
(Ne, β) = (10, 0.97)


...

 22.602304 seconds (184.97 M allocations: 17.124 GiB, 8.47% gc time)
Ne = 600
Ne 600& β 1.04 RMSE: 0.4891095994644669
(Ne, β) = (600, 1.05)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:22


 22.283262 seconds (184.97 M allocations: 17.124 GiB, 9.61% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:13:31


Ne 600& β 1.05 RMSE: 0.4870945374290198


In [11]:
save(path*"metric_enkf.jld", "metric", metric_enkf)

Benchmark for the Lorenz-63 problem with the sequential stochastic radial map filter

p = 0

In [12]:
p = 0

0

In [13]:
metric_srmf0 = benchmark_srmf_lorenz63(model, data, path, Ne_array, β_array, p);

(Ne, β) = (10, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


  5.133800 seconds (29.94 M allocations: 2.060 GiB, 5.57% gc time)
Ne = 10
Ne 10& β 0.95 RMSE: 1.1484494268122272
(Ne, β) = (10, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


  1.813872 seconds (27.44 M allocations: 1.944 GiB, 13.96% gc time)
Ne = 10
Ne 10& β 0.96 RMSE: 0.8844648948939896
(Ne, β) = (10, 0.97)


...

 68.468548 seconds (1.36 G allocations: 94.880 GiB, 12.71% gc time)
Ne = 600
Ne 600& β 1.04 RMSE: 0.4881860190601592
(Ne, β) = (600, 1.05)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:01


 61.724471 seconds (1.36 G allocations: 94.880 GiB, 13.47% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:28:56


Ne 600& β 1.05 RMSE: 0.4856713058195489


In [14]:
save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf0)

p = 1

In [15]:
p = 1

1

In [16]:
metric_srmf1 = benchmark_srmf_lorenz63(model, data, path, [40, 60, 100, 200, 400, 600], β_array, p);

(Ne, β) = (40, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


  7.078899 seconds (110.58 M allocations: 8.314 GiB, 10.37% gc time)
Ne = 40
Ne 40& β 0.95 RMSE: 0.7723808130054851
(Ne, β) = (40, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


  7.025194 seconds (110.57 M allocations: 8.314 GiB, 10.50% gc time)
Ne = 40
Ne 40& β 0.96 RMSE: 0.5629149122349779
(Ne, β) = (40, 0.97)


...

 89.428448 seconds (1.57 G allocations: 119.012 GiB, 10.98% gc time)
Ne = 600
Ne 600& β 1.04 RMSE: 0.3943156740220039
(Ne, β) = (600, 1.05)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:27


 87.424345 seconds (1.57 G allocations: 119.062 GiB, 10.58% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:39:41


Ne 600& β 1.05 RMSE: 0.39856029673685395


In [17]:
save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf1)

p = 2

In [18]:
p = 2

2

In [19]:
metric_srmf2 = benchmark_srmf_lorenz63(model, data, path, [60, 100, 200, 400, 600], collect(0.97:0.01:1.04), p);

(Ne, β) = (60, 0.97)


Progress:  10%|████                                     |  ETA: 0:00:10

Max number of iterations is reached during the optimization

Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


 10.980591 seconds (164.40 M allocations: 12.733 GiB, 10.14% gc time)
Ne = 60
Ne 60& β 0.97 RMSE: 0.762719711537487
(Ne, β) = (60, 0.98)


...

 96.962215 seconds (1.57 G allocations: 120.991 GiB, 10.64% gc time)
Ne = 600
Ne 600& β 1.03 RMSE: 0.37262147782230703
(Ne, β) = (600, 1.04)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:29


 89.806533 seconds (1.56 G allocations: 120.596 GiB, 10.63% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:29:00


Ne 600& β 1.04 RMSE: 0.3840133098523752


In [20]:
save(path*"metric_srmf"*string(p)*".jld", "metric", metric_srmf2)

Benchmark for the Lorenz-63 problem with the sequential stochastic adaptive radial map filter

p = 0

In [21]:
p = 0

0

In [22]:
metric_sadaptivermf0 = benchmark_sadaptivermf_lorenz63(model, data, path, Ne_array, β_array, p);

(Ne, β) = (10, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


  1.974081 seconds (28.33 M allocations: 1.988 GiB, 10.83% gc time)
Ne = 10
Ne 10& β 0.95 RMSE: 0.9566654682239755
(Ne, β) = (10, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


  1.652652 seconds (27.68 M allocations: 1.959 GiB, 11.30% gc time)
Ne = 10
Ne 10& β 0.96 RMSE: 1.1823886359653848
(Ne, β) = (10, 0.97)


...

 62.983135 seconds (1.37 G allocations: 95.738 GiB, 13.00% gc time)
Ne = 600
Ne 600& β 1.04 RMSE: 0.48933039817719837
(Ne, β) = (600, 1.05)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:04


 64.027485 seconds (1.37 G allocations: 95.738 GiB, 12.94% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:30:06


Ne 600& β 1.05 RMSE: 0.4913794081735623


In [23]:
save(path*"metric_sadaptivermf0"*string(p)*".jld", "metric", metric_sadaptivermf0)

p = 1

In [24]:
p = 1

1

In [25]:
metric_sadaptivermf1 = benchmark_sadaptivermf_lorenz63(model, data, path, [40, 60, 100, 200, 400, 600], β_array, p);

(Ne, β) = (40, 0.95)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


  7.295259 seconds (111.13 M allocations: 8.349 GiB, 10.44% gc time)
Ne = 40
Ne 40& β 0.95 RMSE: 0.5816500372451683
(Ne, β) = (40, 0.96)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


  7.343593 seconds (111.14 M allocations: 8.351 GiB, 10.55% gc time)
Ne = 40
Ne 40& β 0.96 RMSE: 0.60686438289528
(Ne, β) = (40, 0.97)


...

 98.496963 seconds (1.57 G allocations: 119.231 GiB, 10.95% gc time)
Ne = 600
Ne 600& β 1.04 RMSE: 0.3926593806082884
(Ne, β) = (600, 1.05)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:28


 88.658584 seconds (1.57 G allocations: 119.161 GiB, 10.93% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:41:19


Ne 600& β 1.05 RMSE: 0.4046611674824419


In [26]:
save(path*"metric_sadaptivermf1"*string(p)*".jld", "metric", metric_sadaptivermf1)

p = 2

In [27]:
p = 2

2

In [28]:
metric_sadaptivermf2 = benchmark_sadaptivermf_lorenz63(model, data, path, [60, 100, 200, 400, 600], collect(0.97:0.01:1.04), p);

(Ne, β) = (60, 0.97)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:11


 11.194188 seconds (165.64 M allocations: 12.854 GiB, 10.06% gc time)
Ne = 60
Ne 60& β 0.97 RMSE: 0.7590563412868957
(Ne, β) = (60, 0.98)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:11


 11.173764 seconds (165.46 M allocations: 12.832 GiB, 10.13% gc time)
Ne = 60
Ne 60& β 0.98 RMSE: 0.8679825855626035
(Ne, β) = (60, 0.99)


...

102.742683 seconds (1.57 G allocations: 120.919 GiB, 10.84% gc time)
Ne = 600
Ne 600& β 1.03 RMSE: 0.3844961954810035
(Ne, β) = (600, 1.04)


Progress: 100%|█████████████████████████████████████████| Time: 0:01:35


 95.489899 seconds (1.57 G allocations: 120.768 GiB, 10.57% gc time)
Ne = 600


Progress: 100%|█████████████████████████████████████████| Time: 0:30:21


Ne 600& β 1.04 RMSE: 0.38634497310726157


In [29]:
save(path*"metric_sadaptivermf2"*string(p)*".jld", "metric", metric_sadaptivermf2)